Lab 9: Tables & Charts
======================
This notebook reviews some ways to make nicely
formatted tables, and then looks at making
interactive bar charts with the
`plotly` library.

00:40 Load Census Data
03:42 Merge Tiger Shapes
05:21 Formatting Tables
10:58 Simple Bar Chart
12:40 Grouped Bar Chart


In [1]:
# install this very alpha library directly from github
!pip install https://github.com/mcuringa/cartopy/raw/refs/heads/main/dist/maptools-latest.tar.gz -q

!pip install census us plotly nbformat -q

from maptools import census_vars
import pandas as pd
import geopandas as gpd
import plotly.express as px
import us
from census import Census

# from google.colab import userdata
# api_key = userdata.get('CENSUS_API_KEY')
import os
api_key = os.getenv('CENSUS_API_KEY')

# creaate one census object we will reuse
c = Census(api_key)


In [2]:
census_vars.search("vehicles")

,group,concept,match
17397,B25044,Tenure By Vehicles Available,62.01%
13675,B08014,Sex Of Workers By Vehicles Available,56.17%
2282,B99085,Allocation Of Vehicles Available For Workers,54.95%
1015,B08201,Household Size By Vehicles Available,53.75%
27120,B25045,Tenure By Vehicles Available By Age Of Householder,51.11%
6073,B25046,Aggregate Number Of Vehicles Available By Tenure,49.91%
24015,B08141,Means Of Transportation To Work By Vehicles Available,49.15%
12548,B08203,Number Of Workers In Household By Vehicles Available,49.05%
27270,B99089,Allocation Of Vehicles Available For Workers For Workplace Geography,43.77%
4296,B08541,Means Of Transportation To Work By Vehicles Available For Workplace Geography,41.78%


Tenure by Vehicle
=================

For our example, we're going to look at Census table **B25044**
_Tenure By Vehicles Available_. The data in this table
looks at the number of vehicles available by household,
and breaks households down by "tenure": whether
the home is owned by the occupants or rented.

In the next few blocks we:
- look up the field names for the table
- copy them into a dict
- rename some fields
- load the data at the county level for NYS
- rename the columns



In [3]:
field_names = census_vars.get_table("B25044")
display(field_names)
field_names = {
    'B25044_001E': 'total_households',
    'B25044_002E': 'owner_occupied',
    'B25044_009E': 'renter_occupied',
    'B25044_003E': 'owner_no_vehicle',
    'B25044_004E': 'owner_1_vehicle',
    'B25044_005E': 'owner_2_vehicles',
    'B25044_006E': 'owner_3_vehicles',
    'B25044_007E': 'owner_4_vehicles',
    'B25044_008E': 'owner_5_or_more_vehicles',
    'B25044_010E': 'renter_no_vehicle',
    'B25044_011E': 'renter_1_vehicle',
    'B25044_012E': 'renter_2_vehicles',
    'B25044_013E': 'renter_3_vehicles',
    'B25044_014E': 'renter_4_vehicles',
    'B25044_015E': 'renter_5_or_more_vehicles'
}

{'B25044_001E': 'total',
 'B25044_002E': 'owner_occupied',
 'B25044_003E': 'owner_occupied_no_vehicle_available',
 'B25044_004E': 'owner_occupied_1_vehicle_available',
 'B25044_005E': 'owner_occupied_2_vehicles_available',
 'B25044_006E': 'owner_occupied_3_vehicles_available',
 'B25044_007E': 'owner_occupied_4_vehicles_available',
 'B25044_008E': 'owner_occupied_5_or_more_vehicles_available',
 'B25044_009E': 'renter_occupied',
 'B25044_010E': 'renter_occupied_no_vehicle_available',
 'B25044_011E': 'renter_occupied_1_vehicle_available',
 'B25044_012E': 'renter_occupied_2_vehicles_available',
 'B25044_013E': 'renter_occupied_3_vehicles_available',
 'B25044_014E': 'renter_occupied_4_vehicles_available',
 'B25044_015E': 'renter_occupied_5_or_more_vehicles_available'}

In [4]:
fields = list(field_names.keys())

data = c.acs5.get(fields=fields, geo={ 'for': 'county:*', 'in': f'state:{us.states.NY.fips}'}, year=2022)

df = pd.DataFrame(data)
df.rename(columns=field_names, inplace=True)
df["statefp"] = df["state"]
df["state"] = df.statefp.apply(census_vars.lookup_state)
df.head(10)

,total_households,owner_occupied,renter_occupied,owner_no_vehicle,owner_1_vehicle,owner_2_vehicles,owner_3_vehicles,owner_4_vehicles,owner_5_or_more_vehicles,renter_no_vehicle,renter_1_vehicle,renter_2_vehicles,renter_3_vehicles,renter_4_vehicles,renter_5_or_more_vehicles,state,county,statefp
0,132175.0,74707.0,57468.0,2656.0,22386.0,34782.0,10655.0,3132.0,1096.0,13821.0,28432.0,12509.0,2113.0,459.0,134.0,NY,001,36
1,16813.0,13584.0,3229.0,605.0,4275.0,5956.0,1929.0,517.0,302.0,620.0,1439.0,917.0,232.0,19.0,2.0,NY,003,36
2,525387.0,104887.0,420500.0,25658.0,46873.0,23467.0,6870.0,1498.0,521.0,290507.0,108030.0,19120.0,2217.0,457.0,169.0,NY,005,36
3,81339.0,52748.0,28591.0,2205.0,17227.0,22700.0,7460.0,2441.0,715.0,7999.0,13802.0,5086.0,1022.0,365.0,317.0,NY,007,36
4,31491.0,23220.0,8271.0,1240.0,7111.0,9961.0,3594.0,987.0,327.0,2049.0,3913.0,1939.0,231.0,63.0,76.0,NY,009,36
5,30910.0,21643.0,9267.0,781.0,6290.0,9816.0,3184.0,1117.0,455.0,2311.0,4727.0,1774.0,281.0,82.0,92.0,NY,011,36
6,53405.0,36891.0,16514.0,1727.0,11617.0,15672.0,5826.0,1321.0,728.0,4575.0,8121.0,3271.0,453.0,57.0,37.0,NY,013,36
7,34779.0,23801.0,10978.0,808.0,7617.0,10423.0,3593.0,1019.0,341.0,2730.0,5356.0,2324.0,501.0,52.0,15.0,NY,015,36
8,19886.0,15170.0,4716.0,534.0,4164.0,6793.0,2373.0,947.0,359.0,1130.0,2219.0,1108.0,235.0,18.0,6.0,NY,017,36
9,32651.0,22542.0,10109.0,808.0,6284.0,10277.0,3779.0,967.0,427.0,2286.0,4558.0,2728.0,351.0,167.0,19.0,NY,019,36


Calculate new columns
--------------------------
In the next block we just calculate car-free households (regardless of tenure)
and the percentages for car-free and car-owning households.

In [5]:
df["carfree"] = df["owner_no_vehicle"] + df["renter_no_vehicle"]
df["carfree_pct"] = df["carfree"] / df["total_households"]

df["carowner"] = df["total_households"] - df["carfree"]
df["carowner_pct"] = df["carowner"] / df["total_households"]

cars = df[["total_households", "carfree", "carfree_pct", "carowner", "carowner_pct", "state", "statefp", "county"]].copy()
cars.head()

,total_households,carfree,carfree_pct,carowner,carowner_pct,state,statefp,county
0,132175.0,16477.0,0.124660,115698.0,0.875340,NY,36,001
1,16813.0,1225.0,0.072860,15588.0,0.927140,NY,36,003
2,525387.0,316165.0,0.601775,209222.0,0.398225,NY,36,005
3,81339.0,10204.0,0.125450,71135.0,0.874550,NY,36,007
4,31491.0,3289.0,0.104443,28202.0,0.895557,NY,36,009


Merge in County Names
---------------------
We're not going to make any maps, but we can find county names in the
tiger county files for New York State. We'll load them by URL and
merge the county name into our data, joining on the fips code.

In [6]:
county_url = "https://www2.census.gov/geo/tiger/TIGER2022/COUNTY/tl_2022_us_county.zip"
counties = gpd.read_file(county_url)
ny_counties = counties[counties.STATEFP == us.states.NY.fips].copy()
ny_counties.rename(columns={"COUNTYFP": "county", "NAME": "county_name"}, inplace=True)
cars = cars.merge(ny_counties[["county", "county_name"]], on="county")
cars.head()

,total_households,carfree,carfree_pct,carowner,carowner_pct,state,statefp,county,county_name
0,132175.0,16477.0,0.124660,115698.0,0.875340,NY,36,001,Albany
1,16813.0,1225.0,0.072860,15588.0,0.927140,NY,36,003,Allegany
2,525387.0,316165.0,0.601775,209222.0,0.398225,NY,36,005,Bronx
3,81339.0,10204.0,0.125450,71135.0,0.874550,NY,36,007,Broome
4,31491.0,3289.0,0.104443,28202.0,0.895557,NY,36,009,Cattaraugus


Style and Formatting
--------------------
Here we use the `style` object to format the table
without altering the underlying data.

`style` has a `format()` method that takes a dictionary
of column names and format strings. This allows us to
format multiple columns at once, without using `apply()`.

We also use the `set_caption()` method to add a title to 
the table, styled with HTML.

Unfortunately, some things aren't possible with `style`, 
so we refactor the code to make a `copy()` of the DataFrame 
so that we can rename columns using our aliases.

In [7]:
formats = {
    "total_households": "{:,.0f}",
    "carfree_pct": "{:,.1%}",
    "carowner_pct": "{:,.1%}",
}
aliases = {
    "county_name": "County",
    "total_households": "Total Households",
    "carfree_pct": "% Car Free",
    "carowner_pct": "% Car Owner",
}

title = "<h3>Car Ownership in New York State Counties</h3>"
cars.sort_values(by="carfree_pct", ascending=False, inplace=True)

cols = ['county_name', 'total_households', 'carfree_pct', 'carowner_pct']
table = cars[cols].head(10).style.format(formats)
# hide the index
table.format_index(lambda x: f"<span style='display: none'>{x}</span>")
table = table.set_caption(title)
table

,county_name,total_households,carfree_pct,carowner_pct
30,New York,"772,601",77.9%,22.1%
2,Bronx,"525,387",60.2%,39.8%
23,Kings,"997,957",55.3%,44.7%
40,Queens,"816,859",36.7%,63.3%
42,Richmond,"170,000",15.2%,84.8%
59,Westchester,"367,740",14.2%,85.8%
54,Tompkins,"42,984",13.3%,86.7%
61,Yates,"8,882",13.0%,87.0%
28,Montgomery,"18,918",12.8%,87.2%
3,Broome,"81,339",12.5%,87.5%


Style a `copy()` of the DataFrame
---------------------------------
Some operations are not possible with the `style` object, 
so we make a copy of the DataFrame and then rename the columns
before using the `style` object to format the table.

Because we called copy(), the original DataFrame is not affected by the
changes we make.
_Note, we re-use `formats` and `title` defined in the previous cell._

In [8]:
aliases = {
    "county_name": "County",
    "total_households": "Total Households",
    "carfree_pct": "% Car Free",
    "carowner_pct": "% Car Owner",
}

formats = {
    "Total Households": "{:,.0f}",
    "% Car Free": "{:,.1%}",
    "% Car Owner": "{:,.1%}",
}

table = cars[aliases.keys()].head(10).copy()
table = table.rename(columns=aliases)

# switch from DataFrame to Styler
table = table.style.format(formats)
table = table.set_caption(title)
table

,County,Total Households,% Car Free,% Car Owner
30,New York,"772,601",77.9%,22.1%
2,Bronx,"525,387",60.2%,39.8%
23,Kings,"997,957",55.3%,44.7%
40,Queens,"816,859",36.7%,63.3%
42,Richmond,"170,000",15.2%,84.8%
59,Westchester,"367,740",14.2%,85.8%
54,Tompkins,"42,984",13.3%,86.7%
61,Yates,"8,882",13.0%,87.0%
28,Montgomery,"18,918",12.8%,87.2%
3,Broome,"81,339",12.5%,87.5%


Making Bar Charts
=================
We are going to use a python library called `plotly` to make
interactive bar charts. We will use the `px.bar()` function.

A few things to note:

- the x-axis labels the values going across the bottom of the chart
- the y-axis labels the values going up the side of the chart
- we're going to work with the `gc` variable, which is a subset of the data
  to make these example charts
- if we sort our data, we will get bard in a different order

A simple horizontal bar chart
------------------------------
With one line of code, we can make a simple horizontal bar chart.

In [9]:
# a basic bar chart showing the number of car-free households in the top 10 counties
fig = px.bar(cars.head(10), x='carfree_pct', y="county_name", 
             title="Top 10 Car Free Counties in NYS")
fig

A multi-column, vertical bar chart
----------------------------------
To create a more complex bar chart, first
we're going to organize a table showing exactly the
data we need for the chart. We're not going to distinguish
renter and owner households, so we add them together
to create new fields. In our bar chart, we want to show
5 categories--the NYC boroughs--and six bars for each
borough.

For each borough, we're going to show the number of households
in each category. We'll even get a little practice with
Python `dict` and `list` objects, and using `for` loops.

In [11]:
# let's get the county fips for NYC
nyc_counties = counties[(counties.STATEFP == us.states.NY.fips) & (counties.NAME.isin(["Bronx", "Kings", "New York", "Queens", "Richmond"]))]
nyc_cars = df.merge(nyc_counties[["COUNTYFP", "NAME"]], left_on="county", right_on="COUNTYFP", how="inner")

boros = {"Bronx": "Bronx", "Kings": "Brooklyn", "New York": "Manhattan",
         "Queens": "Queens", "Richmond": "Staten Island"}

nyc_cars["Borough"] = nyc_cars.NAME.map(boros)

nyc_cars["Zero"] = nyc_cars.owner_no_vehicle + nyc_cars.renter_no_vehicle
nyc_cars["One"] = nyc_cars.owner_1_vehicle + nyc_cars.renter_1_vehicle
nyc_cars["Two"] = nyc_cars.owner_2_vehicles + nyc_cars.renter_2_vehicles
nyc_cars["Three"] = nyc_cars.owner_3_vehicles + nyc_cars.renter_3_vehicles
nyc_cars["Four"] = nyc_cars.owner_4_vehicles + nyc_cars.renter_4_vehicles
nyc_cars["Five +"] = nyc_cars.owner_5_or_more_vehicles + nyc_cars.renter_5_or_more_vehicles

# we will use this to create formats and bars
cols = ["Zero", "One", "Two", "Three", "Four", "Five +"]

# select only the new columns for our table
nyc_cars = nyc_cars[cols + ["Borough"]]

# create an empty dictionary to hold our formats
styles = {}

# for each column, do 2 things:
# - convert it to an integer
# - create a format string for it and put it in the styles dict
for c in cols:
    nyc_cars[c] = nyc_cars[c].astype(int)
    styles[c] = "{:,}"

display(styles)

table = nyc_cars.style.format(styles)
table.set_caption("<h3>Car Ownership in NYC Boroughs</h3>")

{'Zero': '{:,}',
 'One': '{:,}',
 'Two': '{:,}',
 'Three': '{:,}',
 'Four': '{:,}',
 'Five +': '{:,}'}

,Zero,One,Two,Three,Four,Five +,Borough
0,"316,165","154,903","42,587","9,087","1,955",690,Bronx
1,"552,059","350,730","76,825","14,163","2,793","1,387",Brooklyn
2,"601,621","149,874","16,863","2,835",861,547,Manhattan
3,"300,112","329,920","139,665","35,824","8,600","2,738",Queens
4,"25,884","63,820","57,892","16,034","5,020","1,350",Staten Island


In [ ]:
# put some HTML in the title
chart_title = "<b style='color: blue'>Number of Cars Owned by NYC Borough</b>"

fig = px.bar(nyc_cars, 
             x="Borough",
             y=cols,
             labels={"value": "No. Households", "variable": "Vehicles Available"}, # for legend and hover
             title=chart_title)

fig.update_layout(
    barmode='group',
    yaxis_title="Number of Households",
    yaxis_tickformat=',' # use commas to separate thousands
)
fig